# **Homework 4 - Getting to know your customers**
___

Importing packages

In [ ]:
import pandas as pd
import numpy as np

Cleaning the dataset

In [ ]:
data = pd.read_csv('bank_transactions.csv')

In [ ]:
# We use this function to convert TransactionTime in time
def zeros(x):
    l = len(x)
    if l != 6:
        x = '0' * (6-l) + x
    return x

In [ ]:
# We have about 7000 rows with missing values so we can just drop them
data.dropna(inplace=True)

# Convert in datetime this two columns
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

# For TransactionTime we need to split the int in hours, minutes and seconds and convert it in datatime
data.TransactionTime = list(map(str,data.TransactionTime))
data.TransactionTime = list(map(zeros,data.TransactionTime))
data.TransactionTime = pd.to_datetime(data.TransactionTime.apply(lambda x: f'{x[:2]}:{x[2:4]}:{x[4:]}'), format='%H:%M:%S').dt.time

# Adjusting CustomerDOB bringing all the data after 2000 in the previous century and delating all the ones of the 1800
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)
data.drop(data[data.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)

# Converting TransactionAmount in integer
data['TransactionAmount (INR)'] = data['TransactionAmount (INR)'].astype(int)

# Saving the new dataset in pickle format
data.to_pickle('clear_dataset.pkl')

___
### **1. Finding Similar Costumers**

#### 1.1 Set up the data

#### 1.2 Fingerprint hashing

#### 1.3 Locality Sensitive Hashing

___
### **2. Grouping customers together!**

#### 2.1 Getting your data + feature engineering

#### 2.2 Choose your features (variables)!

#### 2.3 Clustering!

#### 2.4 Analysing your results!

___
### **Bonus**

___
### **Command Line Question** 

___
### **Algorithmic Question**